## Introduction and schema

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Overview of the data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting new albums to purchase

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce.

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA. Specifically, we're looking for **which genres sell the most tracks in the USA**.

In [3]:
%%sql

WITH
    customers_usa AS
        (
            SELECT
                il.invoice_line_id,
                il.track_id
            FROM customer c
            INNER JOIN invoice i ON i.customer_id = c.customer_id
            INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
            WHERE c.country = 'USA'   
        )
        
SELECT
    g.name genre,
    COUNT(cs.invoice_line_id) tracks_sold,
    (CAST(COUNT(cs.invoice_line_id) as float)/ (SELECT COUNT(*) FROM customers_usa)) pct_tracks_sold
FROM customers_usa cs
INNER JOIN track t ON t.track_id = cs.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC LIMIT 10;



 * sqlite:///chinook.db
Done.


genre,tracks_sold,pct_tracks_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


From the above, it seems like we should target Punk, Blues, and Pop artists. As such, we'd recommend purchasing albums from Red Tone, Slim Jim Bites, and Meteor and the Girls.

## Analyzing employee sales performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. Let's analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

Specifically, let's find the **total dollar amount of sales for each sales support agent**.

In [4]:
%%sql
SELECT
    e.first_name||" "||e.last_name employee_name,
    e.hire_date,
    SUM(i.total) total_sales,
    (SUM(i.total)/(SELECT SUM(total) FROM invoice)) pct_total_sales
FROM employee e
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1
ORDER BY 3 DESC
LIMIT 10;


 * sqlite:///chinook.db
Done.


employee_name,hire_date,total_sales,pct_total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039,0.3676686987597238
Margaret Park,2017-05-03 00:00:00,1584.0000000000034,0.336346436829936
Steve Johnson,2017-10-17 00:00:00,1393.920000000002,0.29598486441034344


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analyzing sales by country

Now, let's analyze sales by country, looking into the following:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

For countries with only one customer, we'll group these as "Other".

In [5]:
%%sql
WITH
    cust_sales AS
        (
            SELECT
                c.customer_id,
                SUM(i.total) sales_per_cust,
                COUNT(i.invoice_id) invoices_per_cust
            FROM customer c
            INNER JOIN invoice i ON i.customer_id = c.customer_id
            GROUP BY 1
        ),
    total_country AS
        (
            SELECT
                CASE
                    WHEN COUNT(c.customer_id) = 1 THEN "Other"
                    ELSE c.country
                END AS country,
                COUNT(c.customer_id) total_customers,
                SUM(cs.sales_per_cust) total_sales,
                SUM(cs.invoices_per_cust) total_invoices
            FROM customer c
            INNER JOIN cust_sales cs ON cs.customer_id = c.customer_id
            GROUP BY country
        )

SELECT
    country,
    total_customers,
    total_sales,
    (total_sales/total_customers) avg_sales_per_customer,
    (total_sales/total_invoices) avg_order_value
FROM
    (
        SELECT
            tc.*,
            CASE
                WHEN tc.country = 'OTHER' THEN "z"
                ELSE "a"
            END AS sort
        FROM total_country tc
    )
GROUP BY country
ORDER BY sort ASC, total_sales DESC;
    

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.03769230769231,7.942671755725191
Canada,8,535.59,66.94875,7.047236842105264
Brazil,5,427.67999999999995,85.53599999999999,7.011147540983606
France,5,389.07,77.814,7.7814
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24,136.62,9.108
United Kingdom,3,245.51999999999998,81.83999999999999,8.768571428571429
Portugal,2,185.13,92.565,6.3837931034482756
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,1,39.6,39.6,7.92


Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Album versus individual tracks


In [6]:
%%sql

/* this finds the first track of each invoice */
WITH
    invoice_first_track AS
    (
        SELECT
            il.invoice_id invoice_id,
            MIN(il.track_id) first_track_id
        FROM invoice_line il
        GROUP BY 1
    )
    
/* this is a yes or a no */
SELECT
    album_purchase, 
    COUNT(invoice_id) number_of_invoices,
    CAST(COUNT(invoice_id) as float) / (SELECT COUNT(*) FROM invoice) pct_of_invoices

/* this is the data that we're choosing it from */
FROM 
    (
    SELECT
        ifs.*, /* we want to select the invoice ID and the first track, that represents one per invoice. */
        CASE /* in this part, we want to see if those tracks match the album tracks */
            WHEN
                (
                    /* this selects all the tracks from the albums referenced by the FIRST track of the invoices.*/
                    SELECT t.track_id FROM track t 
                    WHERE t.album_id = (
                                        SELECT t2.album_id FROM track t2
                                        WHERE t2.track_id = ifs.first_track_id
                                        )
                    
                    EXCEPT
                    /* select a track from an invoice line when that invoice line belongs to an invoice whose id matches that of the invoice id of the first tracks we selected above.*/
                    SELECT il2.track_id FROM invoice_line il2
                    WHERE il2.invoice_id = ifs.invoice_id
                ) IS NULL
            AND
                (
                    SELECT il2.track_id FROM invoice_line il2
                    WHERE il2.invoice_id = ifs.invoice_id

                    EXCEPT 

                    SELECT t.track_id FROM track t
                    WHERE t.album_id = (
                                        SELECT t2.album_id FROM track t2
                                        WHERE t2.track_id = ifs.first_track_id
                                        ) 
                ) IS NULL
            THEN "yes"
            ELSE "no"
            END AS "album_purchase"
    FROM invoice_first_track ifs
    )
GROUP BY album_purchase;


 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,pct_of_invoices
no,500,0.8143322475570033
yes,114,0.18566775244299674



Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.